In [1]:
import math
from typing import List, Tuple, Dict, Optional

# Define candidate investment schemes and short descriptions
SCHEMES = {
    "Fixed Deposit (FD)": "Low risk, guaranteed returns; good for capital preservation and short/medium term.",
    "Public Provident Fund (PPF)": "Long-term government-backed savings; tax benefits; low risk.",
    "National Pension System (NPS)": "Long-term retirement savings with equity/debt options.",
    "Debt Mutual Funds / Bonds": "Lower volatility than equities; suitable for income and preservation.",
    "Balanced/Hybrid Mutual Funds": "Mix of equity and debt; moderate risk-return profile.",
    "Large-cap Equity Mutual Funds / Index Funds": "Lower volatility among equities; long-term growth.",
    "Mid/Small-cap Equity Mutual Funds": "Higher return potential with higher volatility.",
    "Systematic Investment Plan (SIP) in Mutual Funds": "Method of investing regularly - works well for rupee-cost averaging.",
    "ULIP (Unit Linked Insurance Plan)": "Combines insurance + investment; medium to high charges - suitable if insurance+invest.",
    "Gold (sovereign/ETFs/physical)": "Hedge against inflation and market volatility; diversifier.",
    "Real Estate (Residential/REITs)": "Illiquid but can be good for long-term wealth and rental income.",
    "Emergency Fund (Savings/liquid funds)": "Highly liquid buffer covering 6-12 months expenses.",
}

# Convert risk input to numeric score (original values were likely placeholder/error, normalized to 0..1)
RISK_MAP = {
    "low": 0.2, # Lower risk tolerance
    "medium": 0.5,
    "high": 0.9  # Higher risk tolerance
}

def normalize_text(s: str) -> str:
    """Utility function to clean up string input."""
    return s.strip().lower()

class InvestmentAdvisor:
    def __init__(self):
        # Base suitability weights by scheme for three axes:
        # age_group: 'young' (<35), 'mid' (35-54), 'senior' (>=55)
        # income_tier: 'low' (<500k), 'middle' (500k-2M), 'high' (>2M)
        # risk: continuous 0..1 (higher means more tolerant)
        
        self.base_by_age = {
            "Fixed Deposit (FD)": {"young": 0.3, "mid": 0.4, "senior": 0.9},
            "Public Provident Fund (PPF)": {"young": 0.6, "mid": 0.8, "senior": 0.9},
            "National Pension System (NPS)": {"young": 0.8, "mid": 0.9, "senior": 0.6},
            "Debt Mutual Funds / Bonds": {"young": 0.3, "mid": 0.7, "senior": 0.9},
            "Balanced/Hybrid Mutual Funds": {"young": 0.6, "mid": 0.8, "senior": 0.6},
            "Large-cap Equity Mutual Funds / Index Funds": {"young": 0.9, "mid": 0.8, "senior": 0.4},
            "Mid/Small-cap Equity Mutual Funds": {"young": 0.9, "mid": 0.6, "senior": 0.2},
            "Systematic Investment Plan (SIP) in Mutual Funds": {"young": 0.9, "mid": 0.9, "senior": 0.5},
            "ULIP (Unit Linked Insurance Plan)": {"young": 0.6, "mid": 0.5, "senior": 0.2}, # Original 6.6 corrected to 0.6
            "Gold (sovereign/ETFs/physical)": {"young": 0.4, "mid": 0.6, "senior": 0.6},
            "Real Estate (Residential/REITs)": {"young": 0.3, "mid": 0.8, "senior": 0.7}, # Original 8.7 corrected to 0.7
            "Emergency Fund (Savings/liquid funds)": {"young": 0.9, "mid": 0.9, "senior": 0.9},
        }

        self.base_by_income = {
            "Fixed Deposit (FD)": {"low": 0.8, "middle": 0.7, "high": 0.4},
            "Public Provident Fund (PPF)": {"low": 0.7, "middle": 0.8, "high": 0.6},
            "National Pension System (NPS)": {"low": 0.4, "middle": 0.7, "high": 0.9},
            "Debt Mutual Funds / Bonds": {"low": 0.6, "middle": 0.7, "high": 0.8},
            "Balanced/Hybrid Mutual Funds": {"low": 0.5, "middle": 0.8, "high": 0.8},
            "Large-cap Equity Mutual Funds / Index Funds": {"low": 0.2, "middle": 0.7, "high": 0.9},
            "Mid/Small-cap Equity Mutual Funds": {"low": 0.1, "middle": 0.5, "high": 0.9},
            "Systematic Investment Plan (SIP) in Mutual Funds": {"low": 0.6, "middle": 0.9, "high": 0.9},
            "ULIP (Unit Linked Insurance Plan)": {"low": 0.2, "middle": 0.5, "high": 0.6},
            "Gold (sovereign/ETFs/physical)": {"low": 0.3, "middle": 0.6, "high": 0.8},
            "Real Estate (Residential/REITs)": {"low": 0.1, "middle": 0.6, "high": 0.9},
            "Emergency Fund (Savings/liquid funds)": {"low": 0.95, "middle": 0.9, "high": 0.9},
        }

        # Preference of scheme depending on risk tolerance (r is risk_val from 0 to 1)
        self.risk_sensitivity = {
            "Fixed Deposit (FD)": lambda r: 1 - r, # lower risk (r closer to 0) preferred
            "Public Provident Fund (PPF)": lambda r: 1 - (r * 0.8), # Adjusted from original, favors lower risk
            "National Pension System (NPS)": lambda r: 0.5 + 0.5 * r, # neutral/moderate risk
            "Debt Mutual Funds / Bonds": lambda r: 1 - (r * 0.5), # Adjusted from original, favors lower risk
            "Balanced/Hybrid Mutual Funds": lambda r: 0.6 + 0.4 * r, # favors moderate/higher risk
            "Large-cap Equity Mutual Funds / Index Funds": lambda r: 0.3 + 0.7 * r, # favors higher risk
            "Mid/Small-cap Equity Mutual Funds": lambda r: 0.1 + 0.9 * r, # strongly favors higher risk
            "Systematic Investment Plan (SIP) in Mutual Funds": lambda r: 0.4 + 0.6 * r, # favors higher risk
            "ULIP (Unit Linked Insurance Plan)": lambda r: 0.2 + 0.8 * r, # favors higher risk
            "Gold (sovereign/ETFs/physical)": lambda r: 0.5 + 0.2 * (1 - abs(r - 0.5)), # stableish, favors middle risk
            "Real Estate (Residential/REITs)": lambda r: 0.4 + 0.5 * (1 - abs(r - 0.6)), # neutral/moderate risk
            "Emergency Fund (Savings/liquid funds)": lambda r: 1.0, # always important
        }

    def classify_age_group(self, age: int) -> str:
        """Classifies age into predefined groups."""
        if age < 35:
            return "young"
        elif age < 55:
            return "mid"
        else:
            return "senior"

    def classify_income_tier(self, annual_income: float) -> str:
        """Classifies annual income (INR) into predefined tiers."""
        # low: < 500k, middle: 500k-2M, high: > 2M
        if annual_income < 500_000:
            return "low"
        elif annual_income <= 2_000_000:
            return "middle"
        else:
            return "high"

    def risk_value(self, risk_text: str) -> float:
        """Converts risk text or number (0..1) to a numeric risk tolerance score."""
        s = normalize_text(risk_text)
        
        if s in ("l", "low", "conservative", "safe"):
            return RISK_MAP["low"]
        if s in ("m", "medium", "moderate", "balanced"):
            return RISK_MAP["medium"]
        if s in ("h", "high", "aggressive"):
            return RISK_MAP["high"]
        
        # fallback: try parse number 0..1
        try:
            val = float(risk_text)
            return max(0.0, min(1.0, val))
        except Exception:
            return RISK_MAP["medium"]

    def score_scheme(self, scheme: str, age_group: str, income_tier: str, risk_val: float) -> float:
        """Calculates a combined suitability score for an investment scheme."""
        age_score = self.base_by_age.get(scheme, {}).get(age_group, 0.5)
        income_score = self.base_by_income.get(scheme, {}).get(income_tier, 0.5)
        risk_factor = self.risk_sensitivity.get(scheme, lambda r: 0.5)(risk_val)

        # Emergency fund must always be prioritized
        if scheme == "Emergency Fund (Savings/liquid funds)":
            return 1.0 

        # Combine: weighted geometric mean (using multiplication as a proxy for geometric mean)
        # weights: age 0.35, income 0.25, risk 0.40
        w_age, w_inc, w_risk = 0.35, 0.25, 0.40
        
        # Use multiplicative style (geometric-like)
        # s = (age_score^w_age) * (income_score^w_inc) * (risk_factor^w_risk)
        s = (age_score ** w_age) * (income_score ** w_inc) * (risk_factor ** w_risk)

        return s

    def explain_score(self, scheme: str, age_group: str, income_tier: str, risk_val: float) -> str:
        """Generates an explanation of the raw scores contributing to the scheme score."""
        age_score = self.base_by_age.get(scheme, {}).get(age_group, 0.5)
        income_score = self.base_by_income.get(scheme, {}).get(income_tier, 0.5)
        risk_factor = self.risk_sensitivity.get(scheme, lambda r: 0.5)(risk_val)
        
        return (f"age suitability: {age_score:.2f}, income fit: {income_score:.2f}, "
                f"risk match: {risk_factor:.2f}")

    def suggest(self, age: int, annual_income: float, risk_pref: str, top_n: int = 6) -> List[Dict]:
        """Generates a ranked list of investment suggestions."""
        age_group = self.classify_age_group(age)
        income_tier = self.classify_income_tier(annual_income)
        risk_val = self.risk_value(risk_pref)
        
        scores = []
        for scheme in SCHEMES:
            sc = self.score_scheme(scheme, age_group, income_tier, risk_val)
            scores.append((scheme, sc))

        # Normalise scores for 'confidence' display
        total_raw_score = sum(score for _, score in scores)
        # Avoid division by zero if all scores are 0 (should not happen due to Emergency Fund)
        total_raw_score = total_raw_score or 1.0 
        
        ranked = sorted(scores, key=lambda x: x[1], reverse=True)
        
        result = []
        for scheme, raw_score in ranked[:top_n]:
            confidence = raw_score / total_raw_score
            result.append({
                "scheme": scheme,
                "description": SCHEMES.get(scheme, ""),
                "confidence": round(confidence * 100, 1), # percent
                "raw_score": round(raw_score, 4),
                "explanation": self.explain_score(scheme, age_group, income_tier, risk_val)
            })
        
        return result

# Simple CLI demonstration
def demo_cli():
    print("=== Investment Advisor (Rule-based Expert System) ===")
    try:
        age = int(input("Enter your age (years): ").strip())
        income = float(input("Enter your annual income (INR): ").strip())
        risk = input("Enter your risk preference (low / medium / high): ").strip()
    except Exception as e:
        print("Invalid input. Please enter numeric values for age and income.")
        return

    adv = InvestmentAdvisor()
    suggestions = adv.suggest(age=age, annual_income=income, risk_pref=risk, top_n=6)

    print("\nTop suggestions:\n")
    for i, s in enumerate(suggestions, 1):
        print(f"{i}. {s['scheme']} [{s['confidence']}% confident]")
        print(f"    ({s['description']})")
        print(f"    Reason: {s['explanation']}")
        print()

# Programmatic example to show functionality
if __name__ == "__main__":
    advisor = InvestmentAdvisor()

    # Example 1: Young, medium income, high risk (Age=28, Income=800k INR, Risk=high)
    print("Example A: age=28, income=800000 INR, risk=high")
    out = advisor.suggest(age=28, annual_income=800_000, risk_pref="high")
    for r in out:
        print(f"- {r['scheme']} [{r['confidence']}%] -> {r['explanation']}")
    print("\n")

    # Example 2: Senior, low income, low risk (Age=62, Income=300k INR, Risk=low)
    print("Example B: age=62, income=300000 INR, risk=low")
    out2 = advisor.suggest(age=62, annual_income=300_000, risk_pref="low")
    for r in out2:
        print(f"- {r['scheme']} [{r['confidence']}%] -> {r['explanation']}")
    print("\n")

    # Optional: uncomment the line below to run the interactive CLI
    # demo_cli()

Example A: age=28, income=800000 INR, risk=high
- Emergency Fund (Savings/liquid funds) [12.4%] -> age suitability: 0.90, income fit: 0.90, risk match: 1.00
- Systematic Investment Plan (SIP) in Mutual Funds [11.4%] -> age suitability: 0.90, income fit: 0.90, risk match: 0.94
- Large-cap Equity Mutual Funds / Index Funds [10.6%] -> age suitability: 0.90, income fit: 0.70, risk match: 0.93
- National Pension System (NPS) [10.3%] -> age suitability: 0.80, income fit: 0.70, risk match: 0.95
- Mid/Small-cap Equity Mutual Funds [9.7%] -> age suitability: 0.90, income fit: 0.50, risk match: 0.91
- Balanced/Hybrid Mutual Funds [9.6%] -> age suitability: 0.60, income fit: 0.80, risk match: 0.96


Example B: age=62, income=300000 INR, risk=low
- Emergency Fund (Savings/liquid funds) [14.5%] -> age suitability: 0.90, income fit: 0.95, risk match: 1.00
- Fixed Deposit (FD) [12.1%] -> age suitability: 0.90, income fit: 0.80, risk match: 0.80
- Public Provident Fund (PPF) [11.9%] -> age suitability